<a href="https://colab.research.google.com/github/takky0330/NLP/blob/master/BART_DATA_wikihow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

①wikihow_japaneseを利用した文章要約データセットの取得

In [1]:
### wikihow_japaneseリポジトリをクローン

!git clone https://github.com/Katsumata420/wikihow_japanese.git

Cloning into 'wikihow_japanese'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 59 (delta 17), reused 50 (delta 10), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [ ]:
### 記事をクローリングしてhtmlファイルを取得

# htmlフォルダを作成
%cd /content/wikihow_japanese/
!mkdir -p data/html
# クローリング実行
!bash crawl_article.sh

/content/wikihow_japanese
data/urls/アート・エンタメ.html
data/urls/ビジネス・ファイナンス.html
data/urls/健康.html


In [ ]:
### 取得したhtmlファイルから文章を抽出する

# jsonフォルダを作成
%cd /content/wikihow_japanese/
!mkdir -p data/json
# スクレイピング実行
!bash scrape2jsonl.sh

In [ ]:
### 抽出した文章データをtrain, dev, testの３つに分割する

# outputフォルダを作成
%cd /content/wikihow_japanese/
!mkdir -p data/output
# データ分割実行
!python script/make_data.py data/json data/output data/divided_data.tsv

②スクレイピングによる文章要約データセットの取得方法

In [ ]:
### メインページから19のカテゴリのURLを取得

import requests
from bs4 import BeautifulSoup
import pandas as pd

# wikiHow メインページ
main_url = "https://www.wikihow.jp/%E3%83%A1%E3%82%A4%E3%83%B3%E3%83%9A%E3%83%BC%E3%82%B8"
r = requests.get(main_url)
soup = BeautifulSoup(r.content, "html.parser")

# カテゴリのリスト表示部分の情報を取得する
html_li = soup.find_all("li", attrs={"class": "cat_icon"})

# 条件を指定してその部分だけ情報を取得する
category_title = [li.find("span").text for li in html_li]
base_url = "https://www.wikihow.jp"
urls = [base_url + li.find("a")["href"] for li in html_li]

# 出力用のDataFrameを作成
df = pd.DataFrame(
  list(zip(urls, category_title)),
  columns = ["url", "category_title"]
)

# 出力
df.to_csv("wikihow_category_urls.csv", mode="w", encoding="utf-8")

In [ ]:
### カテゴリページからカテゴリ内のすべての記事のURを重複がないようにスクレイピング

import time
%cd /content
category_df = pd.read_csv("wikihow_category_urls.csv")
!mkdir urls
%cd urls

for url in category_df["url"]:
    try:
        name = category_df["category_title"][category_df['url'] == url].iloc[-1]
        time.sleep(3)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        # カテゴリページの記事一覧部分の情報を取得する
        html_li = soup.find_all("div", attrs={"class": "responsive_thumb"})

        # 条件を指定してその部分だけ情報を取得する
        how_title = [li.find("p").text for li in html_li]
        base_url = "https://www.wikihow.jp"
        urls = [li.find("a")["href"] for li in html_li]

        # 出力用のDataFrameを作成
        df = pd.DataFrame(
          list(zip(urls, how_title)),
          columns = ["url", "how_title"]
        )

        # 重複する記事を削除
        df.drop_duplicates(subset='how_title', keep='last', inplace=True)
        df = df.reset_index(drop=True)

        # 出力
        df.to_csv("wikihow_{}_urls.csv".format(name), mode="w", encoding="utf-8")

    except:
        print("error:" + url)

In [ ]:
### 各記事の本文と要約文を取得

%cd /content
!mkdir wikihow_data

# 文字だけ取得する関数 https://qiita.com/poorko/items/9140c75415d748633a10 参照
def get_text(input_html):
  for script in input_html(["script", "style"]):
    script.decompose()
  text=input_html.get_text()
  lines=[]
  for line in text.splitlines():
    if line and line != "広告" and "smallUrl" not in line and len(line) != 1:
      lines.append(line.strip())
  text="\n".join(line for line in lines)
  return text

category_df = pd.read_csv("wikihow_category_urls.csv")
for category_name in category_df["category_title"]:
    # 念の為、カテゴリごとに5秒待つ
    time.sleep(5)
    how_to_url_df = pd.read_csv("urls/wikihow_{}_urls.csv".format(category_name))
    # カテゴリごとに初期化
    how_tgt = []
    how_src = []
    wiki_title = []
    method_title = []
    method_num = []
    step_num = []
    for url in how_to_url_df["url"]:
      try:
        # サーバーに負荷をかけないために2秒おきに実施する
        time.sleep(2)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        # 記事のタイトルを取得
        how_title = soup.find("h1", attrs={"id": "section_0"}).find("a").text
        # 記事の「方法」部分の情報を取得する
        html_methods = soup.find_all("div", attrs={"class": "steps"})
        # 条件を指定してその部分だけ情報を取得する
        for html_method in html_methods:
          if(len(html_methods) > 1):
            method_number = html_method.find("div", attrs={"class": "altblock"}).find("span").text
            mdehod_title = html_method.find("div", attrs={"class": "altblock"}).next_sibling.next_sibling["id"]
          else:
            # 記事に「方法」が一つしかないとき、スクレイピングが失敗するため以下のようにする。
            method_number = 1
            mdehod_title = how_title

          html_steps = html_method.find_all('li', id=lambda value: value and value.startswith('step-id'))
          step_number= 0
          for step in html_steps:
            step_number+= 1
            src = get_text(step)
            tgt = step.find("b", attrs={"class": "whb"}).text
            how_tgt.append(tgt)
            how_src.append(src)
            wiki_title.append(how_title)
            method_title.append(mdehod_title)
            method_num.append(method_number)
            step_num.append(step_number)
      except:
        print("error：" + url)

    # 出力用のDataFrameを作成
    df = pd.DataFrame(
      list(zip(how_tgt, how_src, wiki_title, method_title, method_num, step_num)),
      columns = ["tgt", "src", "wiki_title", "method_title", "method_num", "step_num"]
    )
    
    # カテゴリごとに出力
    df.to_csv("wikihow_data/wikihow_{}_datasets.csv".format(category_name), mode="w", encoding="utf-8")

In [ ]:
### 取得したデータセットの確認

category_names = [
  "アート・エンタメ",
  "車・乗り物",
  "パソコン・電子機器",
  "学び・コミュニケーション",
  "家族",
  "ビジネス・ファイナンス",
  "食・おもてなし",
  "健康",
  "趣味・DIY", 
  "祝祭日・年中行事",
  "住まいと暮らし・ガーデニング",
  "ビューティー・ファッション",
  "ペット・動物",
  "哲学・宗教",
  "人間関係",
  "スポーツ・フィットネス",
  "旅行",
  "仕事",
  "ティーン",
]

事後処理

In [ ]:
### クリーニング

# 文字列取得の際に削り切れなかった文字をreplaceするなどの前処理を行う関数
def replace_str(df):
  df["tgt"] = df["tgt"].apply(lambda x: str(x).replace('\u3000', '。'))
  df["tgt"] = df["tgt"].apply(lambda x: str(x).replace('出典文献', ''))
  df["tgt"] = df["tgt"].apply(lambda x: str(x).replace('\n', ''))
  df["src"] = df["src"].apply(lambda x: str(x).replace('\u3000', '。'))
  df["src"] = df["src"].apply(lambda x: str(x).replace('出典文献', ''))
  df["src"] = df["src"].apply(lambda x: str(x).replace('\n', ''))
  for i in range(30):
    df["tgt"] = df["tgt"].apply(lambda x: str(x).replace('[{}]'.format(i), ''))
    df["src"] = df["src"].apply(lambda x: str(x).replace('[{}]'.format(i), ''))
  # srcからtgtの文字を取り除く
  df["src"] = df.apply(lambda x: x.src.replace(x.tgt, ''), 1)
  
  return df

# すべてのカテゴリのデータを結合
wikihow_all_df = pd.DataFrame()
for category_name in category_names:
  wikihow_df = pd.read_csv("wikihow_data/wikihow_{}_datasets.csv".format(category_name))
  wikihow_all_df = pd.concat([wikihow_all_df, wikihow_df])

wikihow_all_df = replace_str(wikihow_all_df)

# 文字数を数える
wikihow_all_df["tgt_length"] = wikihow_all_df["tgt"].apply(lambda x: len(str(x)))
wikihow_all_df["src_length"] = wikihow_all_df["src"].apply(lambda x: len(str(x)))

# 出力
wikihow_all_df.to_csv("wikihow_data/wikihow_datasets_by_step.csv".format(category_name), mode="w", encoding="utf-8")

In [ ]:
### wikiHowの「方法」ごとにデータをまとめる

method_group = wikihow_all_df.groupby(["method_title"])
wikihow_by_method_df = pd.DataFrame()
method_tgt = []
method_src = []
wiki_title = []
method_title = []
method_num = []
step_count = []
for name, group in method_group:
  tgt = group["tgt"].to_list()
  tg = ''.join(map(str, tgt)) if len(tgt) > 1 else str(tgt[0])
  method_tgt.append(tg)

  src = group["src"].to_list()
  sr = ''.join(map(str, src)) if len(src) > 1 else str(src[0])
  method_src.append(sr)
  wiki_title.append(group["wiki_title"].to_list()[0])
  method_title.append(group["method_title"].to_list()[0])
  method_num.append(group["method_num"].to_list()[0])
  step_count.append(group["step_num"].to_list()[-1])

wikihow_by_method_df = pd.DataFrame(, 
  list(zip(method_tgt, method_src, wiki_title, method_title, method_num, step_count)),
  columns = ["tgt", "src", "wiki_title", "method_title", "method_num", "step_count"]
)
# 文字数を数える
wikihow_by_method_df["tgt_length"] = wikihow_by_method_df["tgt"].apply(lambda x: len(str(x)))
wikihow_by_method_df["src_length"] = wikihow_by_method_df["src"].apply(lambda x: len(str(x)))

# 出力
wikihow_by_method_df.to_csv("wikihow_data/wikihow_datasets_by_method.csv".format(category_name), mode="w", encoding="utf-8")

In [ ]:
### wikiHowの「タイトル」ごとにまとめる

wiki_group = wikihow_by_method_df.groupby(["wiki_title"])
wikihow_by_wiki_df = pd.DataFrame()
wiki_tgt = []
wiki_src = []
wiki_title = []
method_title_sum = []
method_count = []
step_count = []
for name, group in wiki_group:
  tgt = group["tgt"].to_list()
  tg = ''.join(map(str, tgt)) if len(tgt) > 1 else str(tgt[0])
  wiki_tgt.append(tg)

  src = group["src"].to_list()
  sr = ''.join(map(str, src)) if len(src) > 1 else str(src[0])
  wiki_src.append(sr)

  met = group["method_title"].to_list()
  me = ''.join(map(str, met)) if len(met) > 1 else str(met[0])
  method_title_sum.append(me)

  wiki_title.append(group["wiki_title"].to_list()[0])
  method_count.append(group["method_num"].to_list()[-1])
  step_count.append(sum(group["step_count"].to_list()))

wikihow_by_wiki_df = pd.DataFrame(
  list(zip(method_tgt, method_src, wiki_title, method_title_sum, method_count, step_count)),
  columns = ["tgt", "src", "wiki_title", "method_title_sum", "method_count", "step_count"]
)

# 文字数を数える
wikihow_by_wiki_df["tgt_length"] = wikihow_by_wiki_df["tgt"].apply(lambda x: len(str(x)))
wikihow_by_wiki_df["src_length"] = wikihow_by_wiki_df["src"].apply(lambda x: len(str(x)))
wikihow_by_wiki_df["method_title_sum_length"] = wikihow_by_wiki_df["method_title_sum"].apply(lambda x: len(str(x)))

# 出力
wikihow_by_wiki_df.to_csv("wikihow_data/wikihow_datasets_by_wiki.csv".format(category_name), mode="w", encoding="utf-8")